# III. Multi view/ stereo view geometry

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

from lib import projectpoints

## Epipolar geometry

In [20]:
# Internal parameters of the two cameras
K = np.array([
    [1000, 0   , 300],
    [0   , 1000, 200],
    [0   , 0   , 1  ],
])

# External parameters
R1 = np.eye(3)
t1 = np.zeros((3, 1))

R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
t2 = np.array([[0.2, 2, 1]]).T

### 1. Projection of 3D points on the cameras

In [28]:
def get_distorted(r, distortion_coefficients):
    """Return p(1 + dr(p))."""
    k3, k5, k7 = distortion_coefficients
    n = np.sqrt(r[0, :] ** 2 + r[1, :] ** 2)
    dr = k3 * n ** 2 + k5 * n ** 4 + k7 * n ** 6
    r[:2, :] = r[:2, :] * (1 + dr)
    return r

In [48]:
Q = np.array([[1, 0.5, 4, 1]]).T

q1 = projectpoints(K, R1, t1, Q)[:, 0]
q2 = projectpoints(K, R2, t2, Q)[:, 0]

### 2. Cross product matrix

In [65]:
def CrossOp(p):
    """Return the cross product operator.
    
    Take a vector in 3D and return the 3×3 matrix corresponding
    to taking the cross product with that vector.
    
    Parameter
    ---------
    p: 3 x 1 numpy array
    
    Return
    ------
    CrossOP: 3 x 3 numpy array
        cross product operator.
    
    """
    x, y, z = p.reshape(3)
    return np.array([
        [ 0, - z,  y],
        [ z,   0, -x],
        [-y,   x,  0],
    ])

In [66]:
np.cross(q1.T, q2.T) == CrossOp(q1) @ q2

array([ True,  True,  True])

### 3. Fundamental matrix of the two cameras

In [70]:
def essential_matrix(R, t):
    """Return the essential matrix
    
    Parameters
    ----------
    R: 3 x 3 numpy array
        Rotation matrix of camera 2 in the referential of camera 1
    t: 3 x 1 numpy array
        Translation matrix of camera 2 in the referential of camera 1
    
    Return:
    E: 3 x 3 numpy array
        Essential matrix
    """
    return CrossOp(t) @ R


def fundamental_matrix(R, t, K1, K2):
    """Return the fundamental matrix.
    
    Parameters
    ----------
    K1, K2: 3 x 3 numpy array
        intrinsics parameters of the cameras
    R: 3 x 3 numpy array
        Rotation matrix of camera 2 in the referential of camera 1
    t: 3 x 1 numpy array
        Translation matrix of camera 2 in the referential of camera 1
    
    Return:
    F: 3 x 3 numpy array
        Fundamental matrix
    """
    E = essential_matrix(R, t)
    return np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)

In [91]:
F = fundamental_matrix(R2, t2, K, K)
F

array([[ 3.29311881e-07,  8.19396327e-07,  1.79162592e-03],
       [ 5.15532551e-07, -8.76915984e-07,  9.31426656e-05],
       [-1.29882755e-03,  1.51951700e-03, -1.10072682e+00]])

### 4, 5. Epipolar line

By property of the fundamental matrix:

$q_1 F q_2^T = 0 \Rightarrow l_1 = Fq_2^T$

In [93]:
epipolar_line_1 = F @ q1

$q_2$ has to be on the epipolar line as $q_1 F q_2^T = 0$

In [94]:
q2 @ epipolar_line_1

4.440892098500626e-16

### 6. Relation between the 3D points in world space and in the frame of camera 1
We know that  $\tilde{\mathit{Q}} = \begin{bmatrix}
        \textbf{R}_1 & \textbf{t}_1\\ 0 & 1
    \end{bmatrix} \mathit{Q}$
    
Therefore:
\begin{align*}
    \begin{bmatrix}
        \textbf{R}_1^T & -\textbf{R}_1^T\textbf{t}_1\\ 0 & 1
    \end{bmatrix} \tilde{\mathit{Q}} 
    &=
    \begin{bmatrix}
        \textbf{R}_1^T & -\textbf{R}_1^T\textbf{t}_1\\ 0 & 1
    \end{bmatrix}
    \begin{bmatrix}
        \textbf{R}_1 & \textbf{t}_1\\ 0 & 1
    \end{bmatrix} \mathit{Q}\\
    &= \begin{bmatrix}
        \textbf{I}_3 & \textbf{0}\\ 0 & 1
    \end{bmatrix} \mathit{Q}\\
    &= \mathit{Q}
\end{align*}

# 7. see and understand correction

## Applied epipolar geometry